In [4]:

import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI 
load_dotenv()
llm_model = GoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
res= llm_model.invoke("Explain why sky is blue?")
print(res)

The sky is blue primarily due to a phenomenon called **Rayleigh scattering**. Here's a breakdown:

1.  **Sunlight is White Light:** Sunlight, which appears white to us, is actually made up of all the colors of the rainbow (the visible spectrum: red, orange, yellow, green, blue, indigo, violet). Each color has a different wavelength, with violet and blue having the shortest wavelengths and red having the longest.

2.  **Earth's Atmosphere:** Earth's atmosphere is composed of tiny gas molecules, primarily nitrogen (about 78%) and oxygen (about 21%). These molecules are much smaller than the wavelengths of visible light.

3.  **Rayleigh Scattering:** When sunlight enters Earth's atmosphere, these tiny gas molecules scatter the light. Rayleigh scattering specifically describes what happens when light interacts with particles much smaller than its wavelength.
    *   **Key Principle:** Rayleigh scattering is much more effective for **shorter wavelengths** of light than for longer ones. In f

Streaming

In [6]:
for chunk in llm_model.stream("why sky is blue?"):
    print(chunk, end='')

The sky is blue because of a phenomenon called **Rayleigh scattering**, which describes how light interacts with particles much smaller than its wavelength. Here's a breakdown:

1.  **Sunlight is White Light:** What we perceive as white sunlight is actually made up of all the colors of the rainbow (the visible spectrum: red, orange, yellow, green, blue, indigo, violet). Each color has a different wavelength. Red light has the longest wavelength, and violet light has the shortest.

2.  **Earth's Atmosphere:** Before sunlight reaches our eyes, it has to travel through Earth's atmosphere. The atmosphere is composed of tiny gas molecules (like nitrogen and oxygen) and small particles.

3.  **Scattering of Light:** When sunlight hits these tiny molecules and particles in the atmosphere, it gets scattered in all directions. However, this scattering isn't uniform for all colors.

4.  **Rayleigh Scattering and Wavelengths:**
    *   **Shorter wavelengths (blue and violet light)** are scattered

In [8]:
messages=[("system","you are a helpful assistant that answers any questions"),
          ("human","why sky is blue?")
          ]
res = llm_model.invoke(messages)
print(res)

That's a great question! The reason the sky appears blue is due to a phenomenon called **Rayleigh scattering**, which involves how sunlight interacts with Earth's atmosphere.

Here's a breakdown:

1.  **Sunlight is White Light:** The light from the sun appears white to us, but it's actually made up of all the colors of the rainbow (red, orange, yellow, green, blue, indigo, violet). Each color has a different wavelength.

2.  **Earth's Atmosphere:** Our atmosphere is composed mainly of nitrogen and oxygen molecules, along with tiny dust particles and water vapor.

3.  **Scattering:** When sunlight enters the atmosphere, it collides with these gas molecules and particles. This causes the light to be scattered in all directions.

4.  **Wavelength Matters:**
    *   **Blue and violet light** have shorter wavelengths.
    *   **Red and orange light** have longer wavelengths.

5.  **Rayleigh Scattering:** The tiny nitrogen and oxygen molecules in the atmosphere are much smaller than the wave

Prompt Templates

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a {storyname} story about topic {topic}")
llm_prompt = prompt_template.format(storyname="a liitle girl",topic="horror")
res = llm_model.invoke(llm_prompt)
print(res)

Lily loved her room. It had a window seat piled with fluffy cushions, a rainbow rug, and a shelf full of her favorite stuffed animals. But when the sun went down, and the shadows stretched long and twisty, her room felt… different.

It started with the whispers.

Not loud whispers, just soft, like dry leaves rustling right outside her ear, even when her window was closed tight. Lily, who was seven, would pull her blanket up to her chin and squeeze her eyes shut. "It's just the wind," she'd tell herself, but the wind didn't usually sound like it was trying to spell out her name.

One night, the whispers were clearer. "L-i-l-y…" they seemed to sigh, right from the corner of her room where her big, wooden toy chest sat. Lily held her breath, her heart thumping like a trapped bird. She peeked over her blanket. The toy chest lid was slightly ajar. She was sure she'd closed it.

The next night, it wasn't just whispers. There was a soft, rhythmic *tap-tap-tap*. It came from inside her closet.

In [10]:
# chat prompt template
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a {profession} expert on {topic}"),
        ("human", "Hi {profession} expert, can you please answer a question?"),
        ("ai","sure"),
        ("human","{userinput}")
    ]
    )

messages = chat_template.format_messages(
    profession="Historian",
    topic="kennedy family",
    userinput="How many grandchildren have kenny jr"
    )

res=llm_model.invoke(messages)
print(res)

That's an interesting question, but it seems there might be a slight misunderstanding.

John F. Kennedy Jr. (JFK Jr.) and his wife, Carolyn Bessette-Kennedy, did not have any children. Therefore, he did not have any grandchildren.


Fewshot prompting

In [12]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

example =[
    {"input":"hi","output":"hola"},
    {"input":"bye","output":"adios"}
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=example
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a professional translator to spanish"),
        few_shot_prompt,
        ("human","{input}")
    ]
)



Chains

In [14]:
chain = final_prompt | llm_model
res=chain.invoke({"input":"How are you?"})
print(res)

¿Cómo estás?


Output parsers

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template("Return an json object with 'answer' key for the user question:{question}")

json_parser = SimpleJsonOutputParser()

chain = json_prompt | llm_model | json_parser

res = chain.invoke({"question":"what is the biggest country?"})
print(res)

{'answer': 'Russia'}


Parser using Pydantic

In [18]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

class Joke(BaseModel):
    question: str = Field(description="question to setup the joke")
    answer:str =Field(description="answer to resolve the joke")

parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query. \n{query},\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

chain = prompt | llm_model | parser
res = chain.invoke({"query":"tell me a joke"})
print(res)

{'question': "Why don't scientists trust atoms?", 'answer': 'Because they make up everything!'}


Document Loaders

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("files/test.txt")
loaded_data= loader.load()
print(loaded_data)


ModuleNotFoundError: No module named 'langchain_community'

In [11]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("files/test.pdf")
data = pdf_loader.load_and_split()
print(data)
print(data[0].page_content)

[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2022-02-15T19:32:13+00:00', 'author': 'Katie McCray', 'moddate': '2022-02-15T19:32:13+00:00', 'source': 'files/test.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Getting Started with Data Modules Workshop \nThis document provides the preparations to get the lab environment set up and to access the \nfiles needed to proceed.  \n \n1.1 Log into the Classroom Trial Environment of Cognos Analytics on Cloud \n \nThe instructor has invited you to use an IBM Cloud “On Demand” instance of Cognos \nAnalytics. This is a real cloud instance of Cognos Analytics, the same that you can purchase \nfor your company. The other students will also be accessing this same instance as if you \neach worked for the same company. \n \nFirst, Get your student ID and Password: \n \n1. Check the private chat area of the web session. \n2. Find the student ID and Password that was issued to you by the instr

In [12]:
from langchain_community.document_loaders import WikipediaLoader

wiki = WikipediaLoader(query="Tesla", load_max_docs=1)
data = wiki.load()
print(data)

[Document(metadata={'title': 'Nikola Tesla', 'summary': "Nikola Tesla (10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884, he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Electric in 1888, earned

In [13]:
from langchain_core.prompts import ChatPromptTemplate



chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Answer the {question} for the given context: {context}" )
    ]
)

messages = chat_prompt.format_messages(
    question="what is Tesla?",
    context=data
)

res = llm_model.invoke(messages)
print(res)

content='Based on the provided context, Tesla refers to Nikola Tesla, a Serbian-American engineer, futurist, and inventor, known for his contributions to the design of the modern alternating current (AC) electricity supply system. He is credited with numerous inventions and innovations, including the development of the AC induction motor, polyphase AC patents, and experiments with wireless communication and power distribution.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 1510, 'total_tokens': 1586, 'completion_time': 0.181953376, 'completion_tokens_details': None, 'prompt_time': 0.077177754, 'prompt_tokens_details': None, 'queue_time': 0.052736016, 'total_time': 0.25913113}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b87f2-6d40-7882-bdbc-585ed3dbbe9e-0' tool_calls=[] invalid_tool_calls

CharacterTextSplitter

In [14]:
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False
)

texts = splitter.create_documents([data[0].metadata['summary']])
print(texts)
print(texts[0])
print(texts[1])


[Document(metadata={}, page_content='Nikola Tesla (10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884, he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices'), Document(metadata={}, page_content='With the help of partners to finance and market his ideas, Tesla set up laboratories and

Recursive Character text splitter

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter  = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

texts = splitter.create_documents([data[0].metadata['summary']])
print(texts)
print(len(texts))

[Document(metadata={}, page_content='Nikola Tesla (10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.'), Document(metadata={}, page_content='Born and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884, he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Electr